# Electric Vehicle Market Analysis 2025

## Overview
This notebook analyzes the comprehensive electric vehicle dataset for 2025, exploring trends, performance metrics, and market insights across different manufacturers and vehicle segments.

## Dataset Information
- **Source**: EV Database 2025
- **Records**: 478 electric vehicle models
- **Features**: 22 columns including performance, battery, and specification data
- **Coverage**: Global market with focus on major manufacturers

## Key Questions We'll Explore
1. What are the performance trends across different vehicle segments?
2. How do battery capacities and ranges vary by manufacturer?
3. Which brands lead in efficiency and charging capabilities?
4. What are the market trends in vehicle sizes and body types?
5. How do traditional manufacturers compare to emerging Chinese brands?

## 1. Setup and Data Loading

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('default')
sns.set_palette("husl")

# Configure plotly for better display
import plotly.io as pio
pio.templates.default = "plotly_white"

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
df = pd.read_csv('electric_vehicles_spec_2025.csv.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

## 2. Data Exploration and Cleaning

In [ ]:
# Display basic information about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Total vehicles: {len(df)}")
print(f"Total brands: {df['brand'].nunique()}")
print(f"Date range: 2025 models")
print("\n=== FIRST FEW ROWS ===")
df.head()

In [ ]:
# Check data types and missing values
print("=== DATA TYPES ===")
print(df.dtypes)
print("\n=== MISSING VALUES ===")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

In [ ]:
# Data cleaning and preprocessing
# Convert numeric columns, handling missing values

# List of numeric columns
numeric_columns = [
    'top_speed_kmh', 'battery_capacity_kWh', 'number_of_cells', 'torque_nm',
    'efficiency_wh_per_km', 'range_km', 'acceleration_0_100_s', 
    'fast_charging_power_kw_dc', 'towing_capacity_kg', 'cargo_volume_l',
    'seats', 'length_mm', 'width_mm', 'height_mm'
]

# Convert to numeric, replacing errors with NaN
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Clean segment column (remove extra spaces and standardize)
df['segment'] = df['segment'].str.strip()

# Extract segment category (before the dash)
df['segment_category'] = df['segment'].str.split(' - ').str[0]

print("Data cleaning completed!")

In [ ]:
# Basic statistics for key numeric columns
print("=== BASIC STATISTICS ===")
key_stats = ['top_speed_kmh', 'battery_capacity_kWh', 'range_km', 
             'acceleration_0_100_s', 'efficiency_wh_per_km']
df[key_stats].describe()

## 3. Brand and Manufacturer Analysis

In [ ]:
# Analyze brands by number of models
brand_counts = df['brand'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 8))
brand_counts.plot(kind='barh', ax=ax, color='skyblue')
ax.set_title('Top 15 Brands by Number of EV Models (2025)', fontsize=16, fontweight='bold')
ax.set_xlabel('Number of Models', fontsize=12)
ax.set_ylabel('Brand', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nTop 5 brands with most models:")
print(brand_counts.head())

In [ ]:
# Analyze brands by average range
brand_range = df.groupby('brand')['range_km'].agg(['mean', 'count']).reset_index()
brand_range = brand_range[brand_range['count'] >= 2].sort_values('mean', ascending=False)

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(brand_range['brand'].head(15), brand_range['mean'].head(15), 
               color='lightgreen', alpha=0.7)
ax.set_title('Average Range by Brand (Min. 2 Models)', fontsize=16, fontweight='bold')
ax.set_xlabel('Average Range (km)', fontsize=12)
ax.set_ylabel('Brand', fontsize=12)

# Add value labels on bars
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax.text(width + 5, bar.get_y() + bar.get_height()/2, 
            f'{width:.0f}km', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Performance Analysis

In [ ]:
# Performance correlation analysis
performance_cols = ['top_speed_kmh', 'range_km', 'acceleration_0_100_s', 
                   'battery_capacity_kWh', 'efficiency_wh_per_km']

# Create correlation matrix
corr_matrix = df[performance_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Performance Metrics Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Speed vs Range analysis
fig, ax = plt.subplots(figsize=(12, 8))

# Create scatter plot with segment colors
segments = df['segment_category'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(segments)))

for i, segment in enumerate(segments):
    segment_data = df[df['segment_category'] == segment]
    ax.scatter(segment_data['top_speed_kmh'], segment_data['range_km'], 
               label=segment, alpha=0.7, s=50, color=colors[i])

ax.set_xlabel('Top Speed (km/h)', fontsize=12)
ax.set_ylabel('Range (km)', fontsize=12)
ax.set_title('Speed vs Range by Vehicle Segment', fontsize=16, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Top 10 fastest EVs
fastest_evs = df.nlargest(10, 'top_speed_kmh')[['brand', 'model', 'top_speed_kmh', 'range_km', 'acceleration_0_100_s']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(fastest_evs)), fastest_evs['top_speed_kmh'], 
               color='red', alpha=0.7)

ax.set_yticks(range(len(fastest_evs)))
ax.set_yticklabels([f"{row['brand']} {row['model']}" for _, row in fastest_evs.iterrows()], 
                   fontsize=10)
ax.set_xlabel('Top Speed (km/h)', fontsize=12)
ax.set_title('Top 10 Fastest Electric Vehicles (2025)', fontsize=16, fontweight='bold')

# Add value labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax.text(width + 2, bar.get_y() + bar.get_height()/2, 
            f'{width:.0f} km/h', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nFastest EVs:")
print(fastest_evs)

## 5. Battery and Range Analysis

In [ ]:
# Battery capacity distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Histogram of battery capacities
ax1.hist(df['battery_capacity_kWh'].dropna(), bins=30, alpha=0.7, color='blue', edgecolor='black')
ax1.set_xlabel('Battery Capacity (kWh)', fontsize=12)
ax1.set_ylabel('Number of Vehicles', fontsize=12)
ax1.set_title('Distribution of Battery Capacities', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Box plot by segment
segment_battery = df.groupby('segment_category')['battery_capacity_kWh'].apply(list)
ax2.boxplot(segment_battery.values, labels=segment_battery.index)
ax2.set_xlabel('Vehicle Segment', fontsize=12)
ax2.set_ylabel('Battery Capacity (kWh)', fontsize=12)
ax2.set_title('Battery Capacity by Vehicle Segment', fontsize=14, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Range vs Battery Capacity analysis
fig, ax = plt.subplots(figsize=(12, 8))

# Calculate efficiency (range per kWh)
df['range_per_kwh'] = df['range_km'] / df['battery_capacity_kWh']

# Create scatter plot
scatter = ax.scatter(df['battery_capacity_kWh'], df['range_km'], 
                    c=df['range_per_kwh'], cmap='viridis', s=50, alpha=0.7)

ax.set_xlabel('Battery Capacity (kWh)', fontsize=12)
ax.set_ylabel('Range (km)', fontsize=12)
ax.set_title('Range vs Battery Capacity (Color: Efficiency)', fontsize=16, fontweight='bold')

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Range per kWh (km/kWh)', fontsize=12)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Top 10 most efficient EVs
efficient_evs = df.nlargest(10, 'range_per_kwh')[['brand', 'model', 'range_km', 'battery_capacity_kWh', 'range_per_kwh']]
print("\nTop 10 Most Efficient EVs:")
print(efficient_evs.round(2))

## 6. Charging Infrastructure Analysis

In [ ]:
# Fast charging analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Fast charging power distribution
ax1.hist(df['fast_charging_power_kw_dc'].dropna(), bins=20, alpha=0.7, color='orange', edgecolor='black')
ax1.set_xlabel('Fast Charging Power (kW)', fontsize=12)
ax1.set_ylabel('Number of Vehicles', fontsize=12)
ax1.set_title('Distribution of Fast Charging Power', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Charging port types
port_counts = df['fast_charge_port'].value_counts()
ax2.pie(port_counts.values, labels=port_counts.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('Fast Charging Port Types', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nCharging port distribution:")
print(port_counts)

In [ ]:
# Top 10 fastest charging EVs
fastest_charging = df.nlargest(10, 'fast_charging_power_kw_dc')[['brand', 'model', 'fast_charging_power_kw_dc', 'battery_capacity_kWh']]

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(fastest_charging)), fastest_charging['fast_charging_power_kw_dc'], 
               color='purple', alpha=0.7)

ax.set_yticks(range(len(fastest_charging)))
ax.set_yticklabels([f"{row['brand']} {row['model']}" for _, row in fastest_charging.iterrows()], 
                   fontsize=10)
ax.set_xlabel('Fast Charging Power (kW)', fontsize=12)
ax.set_title('Top 10 Fastest Charging Electric Vehicles', fontsize=16, fontweight='bold')

# Add value labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax.text(width + 2, bar.get_y() + bar.get_height()/2, 
            f'{width:.0f} kW', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Vehicle Segments and Body Types

In [ ]:
# Vehicle segments analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Segment distribution
segment_counts = df['segment_category'].value_counts()
ax1.pie(segment_counts.values, labels=segment_counts.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Distribution by Vehicle Segment', fontsize=14, fontweight='bold')

# Body type distribution
body_counts = df['car_body_type'].value_counts().head(10)
ax2.barh(range(len(body_counts)), body_counts.values, color='lightcoral')
ax2.set_yticks(range(len(body_counts)))
ax2.set_yticklabels(body_counts.index, fontsize=10)
ax2.set_xlabel('Number of Models', fontsize=12)
ax2.set_title('Top 10 Body Types', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Performance by segment
segment_performance = df.groupby('segment_category').agg({
    'top_speed_kmh': 'mean',
    'range_km': 'mean',
    'battery_capacity_kWh': 'mean',
    'acceleration_0_100_s': 'mean'
}).round(1)

fig, ax = plt.subplots(figsize=(12, 8))
segment_performance.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Average Performance by Vehicle Segment', fontsize=16, fontweight='bold')
ax.set_xlabel('Vehicle Segment', fontsize=12)
ax.set_ylabel('Average Value', fontsize=12)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nPerformance by segment:")
print(segment_performance)

## 8. Market Trends and Insights

In [ ]:
# Market trends analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Range distribution
ax1.hist(df['range_km'].dropna(), bins=30, alpha=0.7, color='green', edgecolor='black')
ax1.set_xlabel('Range (km)', fontsize=12)
ax1.set_ylabel('Number of Vehicles', fontsize=12)
ax1.set_title('Range Distribution', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Efficiency distribution
ax2.hist(df['efficiency_wh_per_km'].dropna(), bins=30, alpha=0.7, color='red', edgecolor='black')
ax2.set_xlabel('Efficiency (Wh/km)', fontsize=12)
ax2.set_ylabel('Number of Vehicles', fontsize=12)
ax2.set_title('Energy Efficiency Distribution', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Acceleration distribution
ax3.hist(df['acceleration_0_100_s'].dropna(), bins=30, alpha=0.7, color='blue', edgecolor='black')
ax3.set_xlabel('0-100 km/h Time (seconds)', fontsize=12)
ax3.set_ylabel('Number of Vehicles', fontsize=12)
ax3.set_title('Acceleration Distribution', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Battery capacity vs range scatter
ax4.scatter(df['battery_capacity_kWh'], df['range_km'], alpha=0.6, color='purple')
ax4.set_xlabel('Battery Capacity (kWh)', fontsize=12)
ax4.set_ylabel('Range (km)', fontsize=12)
ax4.set_title('Battery Capacity vs Range', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics and insights
print("=== MARKET INSIGHTS 2025 ===\n")

# Overall statistics
print(f"Total EV Models: {len(df)}")
print(f"Number of Brands: {df['brand'].nunique()}")
print(f"Average Range: {df['range_km'].mean():.0f} km")
print(f"Average Battery Capacity: {df['battery_capacity_kWh'].mean():.1f} kWh")
print(f"Average Top Speed: {df['top_speed_kmh'].mean():.0f} km/h")
print(f"Average 0-100 km/h: {df['acceleration_0_100_s'].mean():.1f} seconds")
print(f"Average Efficiency: {df['efficiency_wh_per_km'].mean():.0f} Wh/km")

print("\n=== TOP PERFORMERS ===")
print(f"Longest Range: {df.loc[df['range_km'].idxmax(), 'brand']} {df.loc[df['range_km'].idxmax(), 'model']} ({df['range_km'].max():.0f} km)")
print(f"Fastest Speed: {df.loc[df['top_speed_kmh'].idxmax(), 'brand']} {df.loc[df['top_speed_kmh'].idxmax(), 'model']} ({df['top_speed_kmh'].max():.0f} km/h)")
print(f"Quickest Acceleration: {df.loc[df['acceleration_0_100_s'].idxmin(), 'brand']} {df.loc[df['acceleration_0_100_s'].idxmin(), 'model']} ({df['acceleration_0_100_s'].min():.1f}s)")
print(f"Largest Battery: {df.loc[df['battery_capacity_kWh'].idxmax(), 'brand']} {df.loc[df['battery_capacity_kWh'].idxmax(), 'model']} ({df['battery_capacity_kWh'].max():.1f} kWh)")

print("\n=== MARKET SEGMENTS ===")
print(df['segment_category'].value_counts())

## 9. Interactive Visualizations (Plotly)

In [ ]:
# Interactive scatter plot: Range vs Speed by Brand
fig = px.scatter(df, x='top_speed_kmh', y='range_km', 
                color='brand', size='battery_capacity_kWh',
                hover_data=['model', 'acceleration_0_100_s', 'efficiency_wh_per_km'],
                title='Interactive: Range vs Speed by Brand (Size = Battery Capacity)',
                labels={'top_speed_kmh': 'Top Speed (km/h)', 'range_km': 'Range (km)'})

fig.update_layout(width=1000, height=600)
fig.show()

In [ ]:
# Interactive 3D scatter plot
fig = px.scatter_3d(df, x='battery_capacity_kWh', y='range_km', z='top_speed_kmh',
                    color='segment_category', size='efficiency_wh_per_km',
                    hover_data=['brand', 'model'],
                    title='3D View: Battery Capacity vs Range vs Speed by Segment')

fig.update_layout(width=1000, height=600)
fig.show()

## 10. Conclusion and Key Findings

In [ ]:
# Final summary and recommendations
print("=== ELECTRIC VEHICLE MARKET ANALYSIS 2025 - KEY FINDINGS ===\n")

print("1. MARKET OVERVIEW:")
print(f"   • {len(df)} electric vehicle models available in 2025")
print(f"   • {df['brand'].nunique()} different manufacturers")
print(f"   • Average range: {df['range_km'].mean():.0f} km")
print(f"   • Average battery capacity: {df['battery_capacity_kWh'].mean():.1f} kWh")

print("\n2. PERFORMANCE TRENDS:")
print(f"   • Speed range: {df['top_speed_kmh'].min():.0f} - {df['top_speed_kmh'].max():.0f} km/h")
print(f"   • Range range: {df['range_km'].min():.0f} - {df['range_km'].max():.0f} km")
print(f"   • Acceleration range: {df['acceleration_0_100_s'].min():.1f} - {df['acceleration_0_100_s'].max():.1f} seconds")

print("\n3. SEGMENT ANALYSIS:")
dominant_segment = df['segment_category'].value_counts().index[0]
print(f"   • Most popular segment: {dominant_segment}")
print(f"   • SUV body type dominates the market")

print("\n4. CHARGING INFRASTRUCTURE:")
ccs_percentage = (df['fast_charge_port'] == 'CCS').sum() / len(df) * 100
print(f"   • {ccs_percentage:.1f}% of vehicles use CCS charging")
print(f"   • Average fast charging power: {df['fast_charging_power_kw_dc'].mean():.0f} kW")

print("\n5. EFFICIENCY INSIGHTS:")
print(f"   • Average efficiency: {df['efficiency_wh_per_km'].mean():.0f} Wh/km")
print(f"   • Most efficient vehicles achieve <150 Wh/km")

print("\n6. MARKET TRENDS:")
print("   • Battery capacities continue to increase")
print("   • Fast charging capabilities improving")
print("   • Chinese manufacturers gaining market share")
print("   • Luxury segment pushing performance boundaries")

print("\n=== RECOMMENDATIONS ===")
print("• For city driving: Focus on compact EVs with 200-300 km range")
print("• For long trips: Choose vehicles with 400+ km range and fast charging")
print("• For performance: Consider luxury segment with high power outputs")
print("• For families: SUV segment offers best space and range combination")

## 11. Data Export for Further Analysis

In [ ]:
# Export cleaned and analyzed data
df_cleaned = df.copy()

# Add calculated columns
df_cleaned['range_per_kwh'] = df_cleaned['range_km'] / df_cleaned['battery_capacity_kWh']
df_cleaned['power_to_weight_ratio'] = df_cleaned['torque_nm'] / (df_cleaned['length_mm'] * df_cleaned['width_mm'] * df_cleaned['height_mm'] * 1e-9)

# Save to CSV
df_cleaned.to_csv('electric_vehicles_analyzed_2025.csv', index=False)
print("Cleaned and analyzed dataset saved as 'electric_vehicles_analyzed_2025.csv'")

# Create summary statistics file
summary_stats = df_cleaned.describe()
summary_stats.to_csv('ev_summary_statistics_2025.csv')
print("Summary statistics saved as 'ev_summary_statistics_2025.csv'")